In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# Load the text file
loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [2]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANCHAIN_API_KEY"] = os.getenv("LANCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [4]:
# Create an embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

db = FAISS.from_documents(docs, embeddings)
db

In [5]:
### querying the vector store
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'

#### As a Retriever

We can also convert the vectorstore into Retriever class. This allows us to easily use it in other LanChain methods, which largely work with retvierers

Retriever can acta like an interface. First we convert the vector store into retriever for query later on

In [7]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower is better.

In [8]:
docs_and_sscore = db.similarity_search_with_score(query)
docs_and_sscore

[(Document(id='43bb30a2-817a-4d9a-a5eb-26ed673b4f0f', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
  np.float32(1.0922663)),
 (Document(id='b8a9631e-07ec-4515-b967-c5

In [9]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[0.004037079401314259,
 -0.04609061777591705,
 -0.009032493457198143,
 -0.007538396865129471,
 -0.016691625118255615,
 0.013326134532690048,
 -0.02384517900645733,
 -0.0010017239255830646,
 -0.04403812065720558,
 0.01565028540790081,
 0.0007965685799717903,
 0.05508538335561752,
 0.002097771968692541,
 -0.0002309765841346234,
 -0.009492795914411545,
 0.01720474846661091,
 -0.013099756091833115,
 0.028221825137734413,
 -0.017355667427182198,
 -0.02828219346702099,
 -0.01296392921358347,
 -0.025792032480239868,
 -0.030998732894659042,
 0.013416686095297337,
 0.01954399235546589,
 -0.017068922519683838,
 -0.00838354229927063,
 0.02467523328959942,
 0.03908798471093178,
 0.040234968066215515,
 -0.006451780907809734,
 0.034832071512937546,
 0.036371443420648575,
 0.016253961250185966,
 -0.005214246455579996,
 -0.005346300546079874,
 0.04062735661864281,
 0.04117066413164139,
 -0.027603058144450188,
 0.00033414378412999213,
 -0.006549878045916557,
 -0.001926101860590279,
 0.00996064394712448

In [10]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='43bb30a2-817a-4d9a-a5eb-26ed673b4f0f', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(id='b8a9631e-07ec-4515-b967-c5b2eb02ba4c', metadata={'sour

In [11]:
### Saving and Loading the Vector Store
db.save_local("faiss_index")

In [13]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
new_db.similarity_search(query)

[Document(id='43bb30a2-817a-4d9a-a5eb-26ed673b4f0f', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(id='b8a9631e-07ec-4515-b967-c5b2eb02ba4c', metadata={'sour